# Process Raw Data

**Output Datasets**:

* vendors.csv - cleaned up vendors data
* customers.parquet - merge of customer meta data & location


The outputs are stored in the `data/processed/` folder

In [32]:
import os
import numpy as np
import pandas as pd

from pathlib import Path

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load All Datasets

In [52]:
orders = pd.read_csv('../data/raw/orders.csv')

tr_locations = pd.read_csv('../data/raw/train_locations.csv')
te_locations = pd.read_csv('../data/raw/test_locations.csv')



tr_customers = pd.read_csv('../data/raw/train_customers.csv')
te_customers = pd.read_csv('../data/raw/test_customers.csv')

sample_sub = pd.read_csv('../data/raw/SampleSubmission.csv')

/Users/renierbotha/anaconda3/envs/zindi-akeed/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15,16,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Clean Vendors

In [87]:
vendors = pd.read_csv('../data/raw/vendors.csv')

vendors.head()

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,OpeningTime2,prepration_time,commission,is_akeed_delivering,discount_percentage,status,verified,rank,language,vendor_rating,sunday_from_time1,sunday_to_time1,sunday_from_time2,sunday_to_time2,monday_from_time1,monday_to_time1,monday_from_time2,monday_to_time2,tuesday_from_time1,tuesday_to_time1,tuesday_from_time2,tuesday_to_time2,wednesday_from_time1,wednesday_to_time1,wednesday_from_time2,wednesday_to_time2,thursday_from_time1,thursday_to_time1,thursday_from_time2,thursday_to_time2,friday_from_time1,friday_to_time1,friday_from_time2,friday_to_time2,saturday_from_time1,saturday_to_time1,saturday_from_time2,saturday_to_time2,primary_tags,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.588596,0.754434,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,-,15,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,08:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,00:00:00,00:30:00,10:00:00,23:59:00,"{""primary_tags"":""4""}",1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.471654,0.744470,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,-,14,0.0,Yes,0.0,1.0,1,11,EN,4.7,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,19:30:00,00:00:00,01:30:00,08:00:00,23:59:00,00:00:00,01:30:00,08:00:00,23:59:00,"{""primary_tags"":""7""}",1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.407527,0.643681,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,-,19,0.0,Yes,0.0,1.0,1,1,EN,4.5,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,08:00:00,22:45:00,NaN,NaN,"{""primary_tags"":""71""}",1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.585385,0.753811,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,-,16,0.0,Yes,0.0,1.0,1,11,EN,4.5,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:30:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,09:00:00,23:45:00,NaN,NaN,"{""primary_tags"":""46""}",1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.480602,0.552850,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,-,10,0.0,Yes,0.0,1.0,1,11,EN,4.4,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,00:30:00,11:00:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,00:01:00,01:30:00,17:45:00,23:59:00,"{""primary_tags"":""32""}",1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


In [88]:
def proc_vendors(vendors):
    drop = [
        'display_orders',
        'city_id',
        'country_id',
        'one_click_vendor',
        'is_akeed_delivering',
        'commission',      
        'language',
        'open_close_flags',
        'vendor_category_id',
        ''
    ]

    def proc_primary_tags(x):
        if not x == 'nan': 
            return eval(x)['primary_tags']
        else:
            return None
        
    
    vendors.drop(columns=drop, inplace=True, errors='ignore')
    
    vendors['primary_tags'] = vendors.primary_tags.astype(str).apply(proc_primary_tags)
    
    vendors['created_at'] = vendors['created_at'].apply(pd.to_datetime)
    vendors['updated_at'] = vendors['updated_at'].apply(pd.to_datetime)

    
    # Get total operating hours for each day of the week
    weekdays = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

    for wd in weekdays:
        vendors[f'{wd}_hours_1'] = (vendors[f'{wd}_to_time1'].astype(np.datetime64) - vendors[f'{wd}_from_time1'].astype(np.datetime64)) / np.timedelta64(1, 'h')
        vendors[f'{wd}_hours_2'] = (vendors[f'{wd}_to_time2'].astype(np.datetime64) - vendors[f'{wd}_from_time2'].astype(np.datetime64)) / np.timedelta64(1, 'h')
        vendors[f'{wd}_op_hours'] = vendors[[f'{wd}_hours_1',f'{wd}_hours_2']].sum(axis=1)
        vendors.drop(columns = [f'{wd}_hours_1',f'{wd}_hours_2', f'{wd}_to_time1', f'{wd}_from_time1', f'{wd}_to_time2', f'{wd}_from_time2'], inplace=True)

    return vendors

In [89]:
vendors = proc_vendors(vendors)

In [90]:
vendors.head()

,id,authentication_id,latitude,longitude,vendor_category_en,delivery_charge,serving_distance,is_open,OpeningTime,OpeningTime2,prepration_time,discount_percentage,status,verified,rank,vendor_rating,primary_tags,vendor_tag,vendor_tag_name,created_at,updated_at,device_type,monday_op_hours,tuesday_op_hours,wednesday_op_hours,thursday_op_hours,friday_op_hours,saturday_op_hours,sunday_op_hours
0,4,118597.0,-0.588596,0.754434,Restaurants,0.0,6.0,1.0,11:00AM-11:30PM,-,15,0.0,1.0,1,11,4.4,4,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",2018-01-30 14:42:04,2020-04-07 15:12:43,3,16.483333,16.483333,16.483333,16.483333,14.483333,14.483333,16.483333
1,13,118608.0,-0.471654,0.744470,Restaurants,0.7,5.0,1.0,08:30AM-10:30PM,-,14,0.0,1.0,1,11,4.7,7,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",2018-05-03 12:32:06,2020-04-05 20:46:03,3,17.483333,17.483333,13.000000,13.000000,17.483333,17.483333,17.483333
2,20,118616.0,-0.407527,0.643681,Restaurants,0.0,8.0,1.0,08:00AM-10:45PM,-,19,0.0,1.0,1,1,4.5,71,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",2018-05-04 22:28:22,2020-04-07 16:35:55,3,14.750000,14.750000,14.750000,14.750000,14.750000,14.750000,14.750000
3,23,118619.0,-0.585385,0.753811,Restaurants,0.0,5.0,1.0,10:59AM-10:30PM,-,16,0.0,1.0,1,11,4.5,46,"5,8,30,24","Burgers,Desserts,Fries,Salads",2018-05-06 19:20:48,2020-04-02 00:56:17,3,14.500000,14.500000,14.500000,14.750000,14.750000,14.750000,14.500000
4,28,118624.0,0.480602,0.552850,Restaurants,0.7,15.0,1.0,11:00AM-11:45PM,-,10,0.0,1.0,1,11,4.4,32,5,Burgers,2018-05-17 22:12:38,2020-04-05 15:57:41,3,13.466667,13.466667,13.466667,13.466667,7.716667,7.716667,13.466667
